In [20]:
import sys, os
from typing import DefaultDict

In [21]:

def parse_bw(filepath, target):
    if target < 0 or target > 5:
        print('Wrong target range!')
        sys.exit(1)
        
    """
    target
    0: msg_size
    1: iter
    2: bw_peak
    3: bw_avg
    4: msg_rate
    5: all
    """
    res = []
    
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            # (msg_size, iteration, bw_peak, bw_avg, msg_rate)
            row = []
            if target == 5:
                row.append(line[0])
                row.append(line[1])
                row.append(line[2])
                row.append(line[3])
                row.append(line[4])
            else:
                row.append(line[target])
            res.append(row)
        except:
            curline += 1
            continue
        curline += 1
    
    return res

def parse_lat(filepath):
    f = open(filepath, "r")
    lines = f.readlines()
    
    startline = 0
    for j in range(len(lines)):
        if lines[j][0:4] == " #by":
            startline = j + 1
            break
    endline = startline
    while endline < len(lines):
        if lines[endline][1:4] == '===':
            break
        endline += 1
        
    curline = startline
    while curline < endline:
        if lines[curline][0:2] == ' --':
            break
        if lines[curline][0] != ' ':
            curline += 1
            continue
        line = lines[curline].split()
        try:            
            return line
                
        except:
            curline += 1
            continue

In [22]:

# Output format = [[ data1 ], [data2], ... , columns = [] ]
dataset = []
columns = ['rnic', 'test_node', 'test_type', 'tx_depth', 'mtu', 'bg_msgsize', 'bg_bw', 'bg_msgrate', 'tar_msgsize', 'tar_bw', 'tar_msgrate']

###################################
## Modify Here
TARGET = 'test_result_bw'
OUTPUT = 'X3Pro_ETH_bw.csv'
###################################

target_dir = './' + TARGET
if os.path.isdir(target_dir) == False:
    print("Directory is not valid!")
    sys.exit()
    
file_list = os.listdir(target_dir)

### Get target files ###
target_list = []
bw_t = ['rb', 'wb', 'sb']
for target in file_list:
    splt_target = target.split('_')
    if splt_target[2] in bw_t and splt_target[7] == 'tf':
        target_list.append(target)

for t in target_list:
    print(t)
    row = []
    splt_t = t.split('_')
    
    # add testing env
    row.append(splt_t[0])
    row.append(splt_t[1])
    row.append(splt_t[2])
    row.append(splt_t[3][1:])
    row.append(splt_t[4][1:])
    
    # add bg_bw
    bg_bw = parse_bw(target_dir + '/' + t[:-2] + 'bf', 5)[0]
    avg_bw = parse_bw(target_dir + '/' + t[:-2] + 'bf', 3)[int(len(bg_bw)/2)][0]
    row.append(bg_bw[0]) # msg size
    row.append(avg_bw) # avg bw
    row.append(bg_bw[4]) # msg rate
    # add tar_bw data
    tar_bw = parse_bw(target_dir + '/' + t, 5)[0]
    row.append(tar_bw[0])
    row.append(tar_bw[3])
    row.append(tar_bw[4])

    print(row)
    dataset.append(row)
    break

print(dataset)

X3ProETH_y306_rb_t1_m512_bg1048576_s4096_tf
['X3ProETH', 'y306', 'rb', '1', '512', '1048576', '3679.27', '0.003826', '4096', '386.71', '0.098997']
[['X3ProETH', 'y306', 'rb', '1', '512', '1048576', '3679.27', '0.003826', '4096', '386.71', '0.098997']]


In [23]:
import pandas as pd

df = pd.DataFrame(dataset, columns=columns)

print(df)
df.to_csv(OUTPUT)

       rnic test_node test_type tx_depth  mtu bg_msgsize    bg_bw bg_msgrate  \
0  X3ProETH      y306        rb        1  512    1048576  3679.27   0.003826   

  tar_msgsize  tar_bw tar_msgrate  
0        4096  386.71    0.098997  
